In [1]:
import datetime
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
import pandas as pd 

In [3]:

WEBDRIVER_PATH = "chromedriver.exe"
DATA = []

# 네이버뉴스 CRAWLER ########################################################################################
def crawler_naverNews(company_name="", srch_keyword=None):
    # LIMIT_NUM 개씩 잘라서 저장
    LIMIT_NUM = 1000
    limit = 300
    seq = 0
    flag = "success"

    # 검색어 리스트 생성 [기업명, 기업명&제품명, 기업명&서비스명, ...]
    srch_keyword_list = [company_name]
    if srch_keyword:
        for k in srch_keyword_list:
            srch_keyword_list.append(str(company_name) + '+' + str(k))
    try:
        webDriver = webdriver.Chrome(executable_path=WEBDRIVER_PATH)
        for keyword in srch_keyword_list:
            NewsData = []

            url = "https://search.naver.com/search.naver?query=" + keyword + "&where=news&ie=utf8&sm=nws_hty"
            webDriver.get(url)
            time.sleep(1)

            # 예외처리: 데이터가 없는경우
            try:
                webDriver.find_element_by_css_selector('div.not_found02')
                print('# naver_news_data # -> NO DATA')
            except Exception as e:
                while True:
                    for news in webDriver.find_elements_by_tag_name('ul.list_news > li.bx'):
                        if len(NewsData) >= LIMIT_NUM:
                            naver_news_data = {
                                "CompanyName": company_name,
                                "DataType": "naver_news",
                                "SearchDate": datetime.datetime.now().strftime("%Y-%m-%d"),
                                "Data": {
                                    'SrchKeyword': keyword,
                                    'NewsData': NewsData
                                }
                            }
                            DATA.append(naver_news_data)
                            # print('# naver_news_data # -> {}'.format(seq))
                            NewsData = []
                        try:
                            result = get_news_text(news)
                            if result:
                                seq += 1
                                tmp = {
                                    "Seq": seq
                                }
                                tmp.update(result)
                                NewsData.append(tmp)
                                # print(tmp)
                            else:
                                continue
                        except Exception as e:
                            print('naver_news_data Error (get news content): {}'.format(e))
                            continue

                    try:
                        # 다음 페이지로 이동가능하다면 이동
                        html = webDriver.find_element_by_tag_name('html')
                        html.send_keys(Keys.END)
                        time.sleep(0.1)
                        css_selector = 'a.btn_next'
                        next_btn = webDriver.find_element_by_tag_name(css_selector)
                        if next_btn.get_attribute('aria-disabled') == "false":
                            if len(NewsData) < limit:
                                print(len(NewsData))
                                next_btn.click()
                                time.sleep(1)
                            else:
                                break

                        else:
                            break
                        
                    except Exception as e:
                        print('naver_news_data Error (move next_page): {}'.format(e))
                        break

            if seq < 1 or NewsData:
                naver_news_data = {
                    "CompanyName": company_name,
                    "DataType": "naver_news",
                    "SearchDate": datetime.datetime.now().strftime("%Y-%m-%d"),
                    "Data": {
                        'SrchKeyword': keyword,
                        'NewsData': NewsData
                    }
                }
                DATA.append(naver_news_data)
                # print('# naver_news_data # -> {}, {}'.format(seq, naver_news_data))

        webDriver.close()
        return flag, DATA

    # 기타 에러 발생
    except Exception as e:
        print('naver_news_data Error: {}'.format(e))
        return "error", None

# 언론사별 html 태그명 가져오기
def get_news_tag_name(press_name):
    tag_name_dict = {
        '이데일리': 'div.news_body',
        '헤럴드경제': 'div.articleView > p',
        '아이뉴스24': 'article.view > p',
        '뉴스1': 'div#articles_detail',
        '전자신문': 'section#articleBody > p',
        '디지털타임스': 'div.art_txt',
        'ZDNet Korea': 'div#articleBody > p',
        '뉴시스': 'div#textBody',
        '데이터넷': 'div#article-view-content-div > p',
        '프라임경제': 'div#news_body_area > p',
        '디지털투데이': 'article#article-view-content-div > p',
        'IT비즈뉴스': 'article#article-view-content-div > p',
        '데일리포스트': 'div#article-view-content-div > p',
        '메트로신문': 'div.col-12 > p',
        '데일리시큐': 'div#article-view-content-div > p',
        '디지털데일리': 'div#news_body_area > div',
        '넥스트데일리': 'div.con_box > p',
        '테크노아': 'td#articleBody > p',
        '아이티데일리': 'article#article-view-content-div',
        '이투데이': 'div.articleView > p',
        '테크M': 'article#article-view-content-div > p',
        '머니투데이': 'div#textBody',
        '뉴스웍스': 'div#articleBody > p',
        '파이낸셜신문': 'div#article-view-content-div > p',
        '데이터뉴스': 'div#news_body_area > div',
        '세계비즈': 'div#article_txt > p',
        '헬로티': 'div#news_body_area > p',
        '연합뉴스': 'div#articleBodyContents',
        '인더스트리뉴스': 'div#article-view-content-div > p',
        '에이빙뉴스': 'div.content > p',
        '조선비즈': 'section.article-body > p',
        '뉴데일리': 'section.nd-news-body.smartOutput > p',
        '비즈니스포스트': 'div.rns_text > div',
        '핀포인트뉴스': 'div.vwc_tcon.detailBody.detailCont',
        '팍스넷뉴스': 'div.content-area.ft-3.trix-content-area > p',
        '블로터': 'div.content',
        '동아일보': 'div.article_txt',
        '중소기업신문': 'article#article-view-content-div > p',
        '글로벌경제': 'article#article-view-content-div',
        '전기신문': 'div#gisa_content > div',
        '비지니스코리아': 'div#article-view-content-div > p',
        '중앙일보': 'div#article_body',
        '시사오늘': 'div#article-view-content-div > p',
        '농업경제신문': 'div.svcon articleContent detailCont',
        '서울파이낸스': 'div#article-view-content-div > p',
        '라이센스뉴스': 'article#article-view-content-div > p',
        '뉴스웨이': 'div#articleText',
        '아시아타임즈': 'div.col-12 > p',
        '포쓰저널': 'article#article-view-content-div > p',
        'EBN': 'div.article > p',
        '여성소비자신문': 'div#articleBody > p',
        '위키트리': 'div#wikicon > p',
        '이코노뉴스': 'article#article-view-content-div > p',
        '매일일보': 'div#article-view-content-div > p',
        '한국경제TV': 'div.art_txt',
        '매일경제': 'div#divNewsContent',
        '한겨레': 'div.text',
        'e대한경제': 'div.text',
        '에너지경제': 'div#news_body_area_contents',
        '한국일보': 'div.col-main.read > p',
        '문화일보': 'div#NewsAdContent',
        'MBN': 'div#newsViewArea',
        'FETV': 'div#news_body_area > p',
        '머니S': 'div#textBody > div > div',
        '아시아경제': 'div#txt_area > p',
        '오피니언뉴스': 'div#article-view-content-div > p',
        '뉴스락': 'div#article-view-content-div > p',
        '일간스포츠': 'div#adiContents',
        '비즈니스워치': 'div.news_body new_editor > p',
        '더팩트': 'div#content_area > p',
        '부산일보': 'div.article_content > p',
        'SBSBiz': 'div#cnbc-front-articleContent-area-font',
        # '블록체인밸리': 'div#article-view-content-div > p',
        '강원도민일보': 'article#article-view-content-div > p',
        '디트뉴스24': 'article#article-view-content-div > p',
        '대전일보': 'div#fontSzArea',
        '투데이에너지': 'div#article-view-content-div > p',
        '강원일보': 'div#kwnews_body'
    }
    if press_name in tag_name_dict.keys():
        return tag_name_dict[press_name]
    else:
        return None

# 기사 본문 내용 긁어오기
def get_news_text(news):
    secondWebDriver = None
    try:
        PressName = news.find_element_by_tag_name('div.info_group > a').text.replace(' ', '')
        tag_name = get_news_tag_name(PressName)
        if tag_name:
            NewsDate = news.find_elements_by_tag_name('div.info_group > span.info')[-1].text
            if '일 전' in NewsDate:
                NewsDate = datetime.datetime.today() - datetime.timedelta(days=int(NewsDate.replace('일 전', '')))
            elif ('시간 전' in NewsDate) or ('분 전' in NewsDate):
                NewsDate = datetime.datetime.today()
            else:
                NewsDate = datetime.datetime.strptime(NewsDate, '%Y.%m.%d.')
            NewsDate = NewsDate.strftime("%Y-%m-%d")
            NewsTitle = news.find_element_by_tag_name('div.news_area > a').text
            url = news.find_element_by_tag_name('div.news_area > a').get_attribute('href')
            secondWebDriver = webdriver.Chrome(executable_path=WEBDRIVER_PATH)

            try:
                secondWebDriver.set_page_load_timeout(20)
                secondWebDriver.get(url)
                NewsContent = [i.text for i in secondWebDriver.find_elements_by_css_selector(tag_name) if i.text not in ['', ' ', '\n']]
                secondWebDriver.close()
                if NewsContent:
                    return {
                        'PressName': PressName,
                        'NewsDate': NewsDate,
                        'NewsTitle': NewsTitle,
                        'NewsContent': NewsContent
                    }
                else:
                    return None
            except:
                secondWebDriver.close()
                return None
        else:
            return None
    except Exception as e:
        if secondWebDriver:
            secondWebDriver.close()
        return None



# main 함수
if __name__ == "__main__":

    DataList = [
"한국엠에스디",
"하이컨시",
"네이버(주)",
"구글코리아",

"한국방송통신대학교",
"아시아나아이디티",
"한국피자헛(유)",
"디지틀조선일보"


    ]

    for idx, company_name in enumerate(DataList):
        print(" ##### {}. {} ##### ".format(idx+1, company_name))
        DATA = []
        flag, result = crawler_naverNews(company_name=company_name)

        # 결과 출력
        if flag == "success":
            print("success")
            for i in range(len(result)):
                data = []
                for j in range(len(result[i]['Data']['NewsData'])):
                    news = []
                    news.append(result[i]['Data']['NewsData'][j]["NewsDate"])
                    news.append(result[i]['Data']['NewsData'][j]["NewsTitle"])
                    news.append(result[i]['Data']['NewsData'][j]["NewsContent"])
                    data.append(news)

                df = pd.DataFrame(data, columns = ["date","title","contents"])
                path = "C:\\Users\\DMC CONET\\Desktop\\개인프로젝트\\데이터\\"
                df.to_csv(path+"{}.csv".format(result[i]['CompanyName']), encoding="utf-8-sig")

 ##### 1. 한국엠에스디 ##### 
1
3
6
7
8
10
11
12
13
14
16
17
18
20
22
24
25
26
27
28
30
33
36
39
42
43
44
47
50
51
51
51
52
53
53
53
54
54
54
54
54
56
57
57
58
60
60
60
60
61
61
63
63
63
63
63
63
63
63
63
64
64
64
64
64
67
69
70
70
72
74
76
77
78
79
80
80
80
80
81
81
81
82
82
83
83
83
83
84
85
86
87
87
87
87
88
88
88
88
88
89
89
89
91
94
95
96
99
103
108
111
111
111
113
115
116
117
120
121
121
123
124
124
124
125
126
126
128
130
133
135
137
141
141
146
146
149
150
152
154
157
158
158
158
161
164
168
170
172
175
178
180
180
181
182
183
184
success
 ##### 2. 하이컨시 ##### 
2
5
8
17
19
21
21
21
23
24
27
28
29
30
33
33
36
38
42
47
success
 ##### 3. 네이버(주) ##### 
2
4
5
7
11
15
18
23
26
29
30
32
34
38
42
46
50
53
53
55
57
59
62
62
64
67
69
71
73
75
75
77
79
81
84
86
90
92
94
96
100
104
109
112
115
117
117
119
121
123
124
124
124
125
126
126
126
126
126
126
126
127
130
132
134
135
135
135
135
138
141
143
143
143
143
144
144
146
149
151
153
156
156
159
163
166
169
170
170
171
171
171
171
171
171
171
17